In [1]:
import numpy as np
import torch
from torch.autograd import Variable

In [2]:
import random

In [3]:
with open('./词性标注.txt',encoding='UTF-8') as f:
    data=f.readlines()[0:1000]

In [4]:
for i in range(len(data)):
    data[i]=data[i][24:]

In [5]:
data_words_labels=[]

In [6]:
for i in range(len(data)):
    if len(data[i])==0:
        continue
    list1=data[i][0:-1].split("  ")
    data_words_labels=data_words_labels+list1   

In [7]:
data_words_labels2=list(set(data_words_labels))

In [8]:
data_Words=[]
data_label=[]

In [9]:
for i in range(len(data_words_labels2)):
    list2=data_words_labels2[i].split("/")
    data_Words.append(list2[0])
    data_label.append(list2[-1])

In [10]:
samp_data_label=list(set(data_label))

In [11]:
num_of_words=len(set(data_Words))

In [12]:
set_data_words=list(set(data_Words))

In [13]:
list_of_word_onehot=[]

In [14]:
for i in range(num_of_words):
    vec=np.zeros(num_of_words)
    vec[i]=1
    list_of_word_onehot.append(vec)

In [15]:
list_of_labels_onehot=[]

In [16]:
for i in range(len(samp_data_label)):
    list_of_labels_onehot.append(i)

In [17]:
dict_labels_to_onehot=dict(zip(samp_data_label,list_of_labels_onehot))

In [18]:
dict_word_to_onehot=dict(zip(set_data_words,list_of_word_onehot))

In [19]:
for i in range(len(data)):
    data[i]=data[i][0:-1]

In [20]:
ran_list=[]

In [21]:
for i in range(50):
    ran_list.append(random.randint(0,1000))

In [22]:
ran_list=list(set(ran_list))

In [23]:
data_for_train=[]
data_for_test=[]

In [24]:
ran_list.sort()

In [25]:
for i in range(len(data)):
    if i in ran_list:
        data_for_test.append(data[i])
    else:
        data_for_train.append(data[i])

In [26]:
def sentence_to_matrix(sentence):
    sentence=sentence[0:-1]
    list_words=sentence.split("  ")
    list_new=list_words[0].split("/")
    matrix_for_words=dict_word_to_onehot[list_new[0]].reshape(1,8132)
    matrix_for_labels=[dict_labels_to_onehot[list_new[-1].strip()]]
    for i in range(1,len(list_words)):
        list_new=list_words[i].split("/")
        
        matrix_for_words=np.vstack((matrix_for_words,dict_word_to_onehot[list_new[0]].reshape(1,8132)))
        matrix_for_labels=matrix_for_labels+[dict_labels_to_onehot[list_new[-1].strip()]]
    return matrix_for_words,matrix_for_labels


        

In [27]:
mat_for_train=[]
mat_for_test=[]

In [28]:
num=0
for i in data_for_train:
 
    if len(i)==0:
        continue
    
    words,labels=sentence_to_matrix(i)
    mat_for_train.append((words,labels))

In [29]:
for i in data_for_test:
    if len(i)==0:
        continue
    
    words,labels=sentence_to_matrix(i)
    mat_for_test.append((words,labels))

tensor([[0.4191, 0.1748, 0.8702,  ..., 0.8389, 0.5127, 0.6536],
        [0.2969, 0.4465, 0.3958,  ..., 0.7848, 0.5995, 0.1338],
        [0.5605, 0.8033, 0.2301,  ..., 0.2197, 0.3952, 0.0488],
        ...,
        [0.6744, 0.6846, 0.2045,  ..., 0.2560, 0.8135, 0.2139],
        [0.8684, 0.4770, 0.1912,  ..., 0.7969, 0.9659, 0.6693],
        [0.3365, 0.2814, 0.4803,  ..., 0.8751, 0.6480, 0.1884]],
       dtype=torch.float64)

In [207]:
w1=Variable(torch.rand(8132,47,dtype=torch.float64),requires_grad=True)
w2=Variable(torch.rand(47,47,dtype=torch.float64),requires_grad=True)
learning_rate=1e-4

In [208]:
def CRF(n,input_dataset,labels):
    input_dataset=torch.from_numpy(input_dataset)
    input_dataset=Variable(input_dataset,requires_grad=False)
    Z=(input_dataset[0].reshape(1,8132)).mm(w1)
    sum_exp=Z[0,labels[0]]
    for i in range(n-1):
        x=torch.zeros(1,47,dtype=float)
        x[0,np.argmax(labels[i+1])]=1
        score1=x.mm(w2)[0,labels[i+1]]
        score2=(input_dataset[i+1].reshape(1,8132)).mm(w1)[0,labels[i+1]]
        sum_exp=sum_exp+score1+score2
    up_score=sum_exp
    
    
    for i in range(n-1):
        x=torch.zeros(1,47,dtype=float)
        x[0,np.argmax(labels[i+1])]=1
        score1=x.mm(w2)
        score2=(input_dataset[i+1].reshape(1,8132)).mm(w1)
        Z=Z+score1+score2
    all_Z=torch.sum(Z)
    
    p=-up_score/all_Z
    

    loss=p
    
    loss.backward()
    
    w1.data-=learning_rate*w1.grad.data
    w2.data-=learning_rate*w2.grad.data
        
    w1.grad.data.zero_()
    w2.grad.data.zero_()

In [209]:
def train(epochs,all_data):
    for i in range(epochs):
        #print(w1)
        num=0
        print("\r总进度完成%.2f %%" % (i *100 /epochs), end="")
        print()
        for input_dataset,labels in all_data:
            num+=1
            print("\r子进度完成%.2f %%" % (num *100 /len(all_data)), end="")

            
            CRF(len(input_dataset),input_dataset,labels)
        print()

    return w1,w2


In [210]:
mat_for_train[0]

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 [22, 22, 45, 44, 46, 45, 43, 38, 38, 45, 43, 22, 45, 35, 24, 43])

In [211]:
w1,w2=train(10,mat_for_train)

总进度完成0.00 %
子进度完成100.00 %
总进度完成10.00 %
子进度完成100.00 %
总进度完成20.00 %
子进度完成100.00 %
总进度完成30.00 %
子进度完成100.00 %
总进度完成40.00 %
子进度完成100.00 %
总进度完成50.00 %
子进度完成100.00 %
总进度完成60.00 %
子进度完成100.00 %
总进度完成70.00 %
子进度完成100.00 %
总进度完成80.00 %
子进度完成100.00 %
总进度完成90.00 %
子进度完成100.00 %


In [237]:
def find_best(input_dataset,n,true_labels):
    input_dataset=torch.from_numpy(input_dataset)

    input_dataset=Variable(input_dataset,requires_grad=False)
    list_Z=[]
    
    G=w2
    Z=(input_dataset[0].reshape(1,8132)).mm(w1)
    list_Z.append(Z)
    for i in range(n-1):
        Z=(Z.mm(G)).mul(((input_dataset[i+1].reshape(1,8132)).mm(w1)))
        list_Z.append(Z)
    
    
    list_label=[]
    
    for i in range(len(list_Z)):
        list_label.append(true_labels[list(list_Z[i].detach().numpy()[0]).index(max(list(list_Z[i].detach().numpy()[0])))])
    return list_label
        
    

In [238]:
def test(input_dataset,samp_data_label):
    list_label=find_best(input_dataset[0],len(input_dataset[0]),samp_data_label)
    num1=0
    #num1=sum(np.array(input_dataset[1])==np.array(list_label))
    num2=len(input_dataset[1])
    for i in range(len(list_label)):
        if list_label[i]==samp_data_label[input_dataset[1][i]]:
            num1+=1
        
    
    return num1,num2

In [239]:
sum1=0
sum2=0
for i in mat_for_test:
    num1,num2=test(i,samp_data_label)
    sum1+=num1
    sum2+=num2
print(sum1/sum2)

0.04043351396415173
